# Imports

In [1]:
from torch.utils.data import Dataset, DataLoader
from typing import List, Callable, Union
import numpy as np
import torch
import glob
import random
import os

%load_ext nb_black

<IPython.core.display.Javascript object>

# Download and process data

In [2]:
# Run shell script that takes a .txt file with PDBIDs as input.
!./download_and_process_data.sh data/pdbids_010.txt

Successfully downloaded 4X2U.pdb to data/pdbs/raw/4X2U.pdb. 1/10.
Successfully downloaded 2X96.pdb to data/pdbs/raw/2X96.pdb. 2/10.
Successfully downloaded 4MXD.pdb to data/pdbs/raw/4MXD.pdb. 3/10.
Successfully downloaded 3E9L.pdb to data/pdbs/raw/3E9L.pdb. 4/10.
Successfully downloaded 1UWC.pdb to data/pdbs/raw/1UWC.pdb. 5/10.
Successfully downloaded 4BGU.pdb to data/pdbs/raw/4BGU.pdb. 6/10.
Successfully downloaded 2YSW.pdb to data/pdbs/raw/2YSW.pdb. 7/10.
Successfully downloaded 4OW4.pdb to data/pdbs/raw/4OW4.pdb. 8/10.
Successfully downloaded 2V5E.pdb to data/pdbs/raw/2V5E.pdb. 9/10.
Successfully downloaded 1IXH.pdb to data/pdbs/raw/1IXH.pdb. 10/10.
Successfully cleaned data/pdbs/raw/1IXH.pdb and added it to data/pdbs/cleaned/. 1/10.
Successfully cleaned data/pdbs/raw/1UWC.pdb and added it to data/pdbs/cleaned/. 2/10.
Successfully cleaned data/pdbs/raw/2V5E.pdb and added it to data/pdbs/cleaned/. 3/10.
Successfully cleaned data/pdbs/raw/2X96.pdb and added it to data/pdbs/cleaned/. 4

<IPython.core.display.Javascript object>

# Global variables

In [3]:
DEVICE = "cuda"  # "cpu" or "cuda"
BATCH_SIZE = 100
LEARNING_RATE = 3e-4
EPOCHS = 5
TRAIN_VAL_SPLIT = 0.8

<IPython.core.display.Javascript object>

# Data classes

In [4]:
class ResidueEnvironment:
    """
    Residue environment class used to hold necessarry information about the
    atoms of the environment such as atomic coordinates, atom types and the
    class of the missing central amino acid.

    Parameters
    ----------
    xyz_coords: np.ndarray
        Numpy array with shape (n_atoms, 3) containing the x, y, z coordinates.
    atom_types: np.ndarray
        1D numpy array containing the atom types. Integer values in range(6).
    restypes_onehot: np.ndarray
        Numpy array with shape (n_atoms, 21) containing the amino acid
        class of the missing amino acid
    chain_id: str
        Chain id associated to ResidueEnvironment object
    pdb_residue_number: int
        Residue number associated with the ResidueEnvironment object
    pdb_id: str
        PDBID associated with the ResidueEnvironment object
    """

    def __init__(
        self,
        xyz_coords: np.ndarray,
        atom_types: np.ndarray,
        restypes_onehot: np.ndarray,
        chain_id: str,
        pdb_residue_number: int,
        pdb_id: str,
    ):
        self._xyz_coords = xyz_coords
        self._atom_types = atom_types
        self._restypes_onehot = restypes_onehot
        self._chain_id = chain_id
        self._pdb_residue_number = pdb_residue_number
        self._pdb_id = pdb_id

    @property
    def xyz_coords(self):
        return self._xyz_coords

    @property
    def atom_types(self):
        return self._atom_types

    @property
    def restypes_onehot(self):
        return self._restypes_onehot

    @property
    def chain_id(self):
        return self._chain_id

    @property
    def pdb_residue_number(self):
        return self._pdb_residue_number

    @property
    def pdb_id(self):
        return self._pdb_id

    def __repr__(self):
        return (
            f"<ResidueEnvironment with {self.xyz_coords.shape[0]} atoms. \n"
            f"pdb_id: {self.pdb_id}, "
            f"chain_id: {self.chain_id}, "
            f"pdb_residue_number: {self.pdb_residue_number},\n"
            f"restype_one: {np.argmax(self.restypes_onehot)}>"
        )


class ResidueEnvironmentsDataset(Dataset):
    """
    Residue environment dataset class

    Parameters
    ----------
    input_data: Union[List[str], List[ResidueEnvironment]]
        List of parsed pdb filenames in .npz format or list of
        ResidueEnvironment objects
    transform: Callable
        A to-tensor transformer class
    """

    def __init__(
        self,
        input_data: Union[List[str], List[ResidueEnvironment]],
        transformer: Callable = None,
    ):
        if all(isinstance(x, ResidueEnvironment) for x in input_data):
            self._res_env_objects = input_data
        elif all(isinstance(x, str) for x in input_data):
            self._res_env_objects = self._parse_envs(input_data)
        else:
            raise ValueError(
                "Input data is not of type" "Union[List[str], List[ResidueEnvironment]]"
            )

        self.transformer = transformer

    @property
    def res_env_objects(self):
        return self._res_env_objects

    @property
    def transformer(self):
        return self._transformer

    @transformer.setter
    def transformer(self, transformer):
        """TODO: Think if a constraint to add later"""
        self._transformer = transformer

    def __len__(self):
        return len(self.res_env_objects)

    def __getitem__(self, idx):
        sample = self.res_env_objects[idx]
        if self.transformer:
            sample = self.transformer(sample)
        return sample

    def _parse_envs(self, npz_filenames: List[str]) -> List[ResidueEnvironment]:
        """
        TODO: Make this more readable
        """

        res_env_objects = []
        for i in range(len(npz_filenames)):
            coordinate_features = np.load(npz_filenames[i])
            atom_coords_prot_seq = coordinate_features["positions"]
            restypes_onehots_prot_seq = coordinate_features["aa_onehot"]
            selector_prot_seq = coordinate_features["selector"]
            atom_types_flattened = coordinate_features["atom_types_numeric"]

            chain_ids = coordinate_features["chain_ids"]
            pdb_residue_numbers = coordinate_features["residue_numbers"]
            chain_boundary_indices = coordinate_features["chain_boundary_indices"]

            pdb_id = os.path.basename(npz_filenames[i])[0:4]

            N_residues = selector_prot_seq.shape[0]
            for resi_i in range(N_residues):
                selector = selector_prot_seq[resi_i]
                selector_masked = selector[selector > -1]  # Remove Filler
                coords_mask = (
                    atom_coords_prot_seq[resi_i, :, 0] != -99.0
                )  # Remove filler
                coords = atom_coords_prot_seq[resi_i][coords_mask]
                atom_types = atom_types_flattened[selector_masked]
                restypes_onehot = restypes_onehots_prot_seq[resi_i]

                pdb_residue_number = int(pdb_residue_numbers[resi_i])
                # Locate chain id
                for j in range(len(chain_ids)):
                    chain_boundary_0 = chain_boundary_indices[j]
                    chain_boundary_1 = chain_boundary_indices[j + 1]
                    if resi_i in range(chain_boundary_0, chain_boundary_1):
                        chain_id = str(chain_ids[j])
                        break

                res_env_objects.append(
                    ResidueEnvironment(
                        coords,
                        atom_types,
                        restypes_onehot,
                        chain_id,
                        pdb_residue_number,
                        pdb_id,
                    )
                )

        return res_env_objects


class ToTensor:
    """ To-tensor transformer class"""

    def __call__(self, sample: ResidueEnvironment):
        """Converts single ResidueEnvironment object into x_ and y_"""

        sample_env = np.hstack(
            [np.reshape(sample.atom_types, [-1, 1]), sample.xyz_coords]
        )

        return {
            "x_": torch.tensor(sample_env, dtype=torch.float32).to(DEVICE),
            "y_": torch.tensor(
                np.array(sample.restypes_onehot), dtype=torch.float32
            ).to(DEVICE),
        }

    @staticmethod
    def collate_cat(batch: List[ResidueEnvironment]):
        """
        Collate method used by the dataloader to collate a
        batch of ResidueEnvironment objects.
        """
        target = torch.cat([torch.unsqueeze(b["y_"], 0) for b in batch], dim=0)

        # To collate the input, we need to add a column which
        # specifies the environtment each atom belongs to
        env_id_batch = []
        for i, b in enumerate(batch):
            n_atoms = b["x_"].shape[0]
            env_id_arr = torch.zeros(n_atoms, dtype=torch.float32).to(DEVICE) + i
            env_id_batch.append(
                torch.cat([torch.unsqueeze(env_id_arr, 1), b["x_"]], dim=1)
            )
        data = torch.cat(env_id_batch, dim=0)

        return data, target

<IPython.core.display.Javascript object>

# Model class

In [5]:
class CavityModel(torch.nn.Module):
    """
    3D convolutional neural network to missing amino acid classification

    Parameters
    ----------
    n_atom_types: int
        Number of atom types. (C, H, N, O, S, P)
    bins_per_angstrom: float
        Number of grid points per Anstrom.
    grid_dim: int
        Grid dimension
    sigma: float
        Standard deviation used for gaussian blurring
    """

    def __init__(
        self,
        n_atom_types: int = 6,
        bins_per_angstrom: float = 1.0,
        grid_dim: int = 18,
        sigma: float = 0.6,
    ):

        super().__init__()

        self._n_atom_types = n_atom_types
        self._bins_per_angstrom = bins_per_angstrom
        self._grid_dim = grid_dim
        self._sigma = sigma

        self._model()

    @property
    def n_atom_types(self):
        return self._n_atom_types

    @property
    def bins_per_angstrom(self):
        return self._bins_per_angstrom

    @property
    def grid_dim(self):
        return self._grid_dim

    @property
    def sigma(self):
        return self._sigma

    @property
    def sigma_p(self):
        return self.sigma * self.bins_per_angstrom

    @property
    def lin_spacing(self):
        lin_spacing = np.linspace(
            start=-self.grid_dim / 2 * self.bins_per_angstrom
            + self.bins_per_angstrom / 2,
            stop=self.grid_dim / 2 * self.bins_per_angstrom
            - self.bins_per_angstrom / 2,
            num=self.grid_dim,
        )
        return lin_spacing

    def _model(self):
        self.xx, self.yy, self.zz = torch.tensor(
            np.meshgrid(
                self.lin_spacing, self.lin_spacing, self.lin_spacing, indexing="ij"
            ),
            dtype=torch.float32,
        ).to(DEVICE)

        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv3d(6, 16, kernel_size=(3, 3, 3), stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm3d(16),
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv3d(16, 32, kernel_size=(3, 3, 3), stride=2, padding=0),
            torch.nn.ReLU(),
            torch.nn.BatchNorm3d(32),
        )
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv3d(32, 64, kernel_size=(3, 3, 3), stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm3d(64),
            torch.nn.Flatten(),
        )
        self.dense1 = torch.nn.Sequential(
            torch.nn.Linear(in_features=4096, out_features=128),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(128),
        )
        self.dense2 = torch.nn.Linear(in_features=128, out_features=21)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self._gaussian_blurring(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

    def _gaussian_blurring(self, x: torch.Tensor) -> torch.Tensor:
        """
        Method that takes 2d torch.Tensor describing the atoms of the batch.

        Parameters
        ----------
        x: torch.Tensor
            Tensor for shape (n_atoms, 5). Each row represents an atom, where:
                column 0 describes the environment of the batch the
                atom belongs to
                column 1 describes the atom type
                column 2,3,4 are the x, y, z coordinates, respectively

        Returns
        -------
        fields_torch: torch.Tensor
            Represents the structural environment with gaussian blurring
            and has shape (-1, self.grid_dim, self.grid_dim, self.grid_dim).
        """
        current_batch_size = torch.unique(x[:, 0]).shape[0]
        fields_torch = torch.zeros(
            (
                current_batch_size,
                self.n_atom_types,
                self.grid_dim,
                self.grid_dim,
                self.grid_dim,
            )
        ).to(DEVICE)
        for j in range(self.n_atom_types):
            mask_j = x[:, 1] == j
            atom_type_j_data = x[mask_j]
            if atom_type_j_data.shape[0] > 0:
                pos = atom_type_j_data[:, 2:]
                density = torch.exp(
                    -(
                        (torch.reshape(self.xx, [-1, 1]) - pos[:, 0]) ** 2
                        + (torch.reshape(self.yy, [-1, 1]) - pos[:, 1]) ** 2
                        + (torch.reshape(self.zz, [-1, 1]) - pos[:, 2]) ** 2
                    )
                    / (2 * self.sigma_p ** 2)
                )

                # Normalize each atom to 1
                density /= torch.sum(density, dim=0)

                # Since column 0 of atom_type_j_data is sorted
                # I can use a trick to detect the boundaries based
                # on the change from one value to another.
                change_mask_j = (
                    atom_type_j_data[:, 0][:-1] != atom_type_j_data[:, 0][1:]
                )

                # Add begin and end indices
                ranges_i = torch.cat(
                    [
                        torch.tensor([0]),
                        torch.arange(atom_type_j_data.shape[0] - 1)[change_mask_j] + 1,
                        torch.tensor([atom_type_j_data.shape[0]]),
                    ]
                )

                # Fill tensor
                for i in range(ranges_i.shape[0]):
                    if i < ranges_i.shape[0] - 1:
                        index_0, index_1 = ranges_i[i], ranges_i[i + 1]
                        fields = torch.reshape(
                            torch.sum(density[:, index_0:index_1], dim=1),
                            [self.grid_dim, self.grid_dim, self.grid_dim],
                        )
                        fields_torch[i, j, :, :, :] = fields
        return fields_torch

<IPython.core.display.Javascript object>

# Parse and train/val split

In [6]:
parsed_pdb_filenames = sorted(glob.glob("data/pdbs/parsed/*coord*"))
random.shuffle(parsed_pdb_filenames)

n_train_pdbs = int(len(parsed_pdb_filenames) * TRAIN_VAL_SPLIT)
filenames_train = parsed_pdb_filenames[:n_train_pdbs]
filenames_val = parsed_pdb_filenames[n_train_pdbs:]

dataset_train = ResidueEnvironmentsDataset(filenames_train, transformer=ToTensor())
dataset_val = ResidueEnvironmentsDataset(filenames_val, transformer=ToTensor())

dataloader_train = DataLoader(
    dataset_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=ToTensor.collate_cat,
    drop_last=True,
)
dataloader_val = DataLoader(
    dataset_val,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=ToTensor.collate_cat,
    drop_last=True,
)

print(
    f"Training data set includes {len(filenames_train)} pdbs with "
    f"{len(dataset_train)} environments."
)
print(
    f"Validation data setincludes {len(filenames_val)} pdbs with "
    f"{len(dataset_val)} environments."
)

Training data set includes 8 pdbs with 3725 environments.
Validation data setincludes 2 pdbs with 1533 environments.


<IPython.core.display.Javascript object>

# Train

In [7]:
def _train_step(
    cavity_model: CavityModel,
    optimizer: torch.optim.Adam,
    loss_function: torch.nn.CrossEntropyLoss,
) -> (torch.Tensor, float):
    cavity_model.train()
    optimizer.zero_grad()
    batch_y_pred = cavity_model(batch_x)
    loss_batch = loss_function(batch_y_pred, torch.argmax(batch_y, dim=-1))
    loss_batch.backward()
    optimizer.step()
    return (batch_y_pred, loss_batch.detach().cpu().item())


# Define model
cavity_model = CavityModel().to(DEVICE)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cavity_model.parameters(), lr=LEARNING_RATE)

# Train loop
for epoch in range(EPOCHS):
    loss_running_mean = 0.0
    labels_true = []
    labels_pred = []
    for batch_x, batch_y in dataloader_train:
        # Take train step
        batch_y_pred, loss_batch = _train_step(cavity_model, optimizer, loss_function)

        # Exponential running mean for the loss
        loss_running_mean = loss_running_mean * 0.9 + loss_batch * 0.1

        labels_true.append(torch.argmax(batch_y, dim=-1).detach().cpu().numpy())
        labels_pred.append(torch.argmax(batch_y_pred, dim=-1).detach().cpu().numpy())
    acc_train = np.mean((np.reshape(labels_true, -1) == np.reshape(labels_pred, -1)))

    # Eval loop. Due to memory, we don't pass the whole data set to the model
    labels_true_val = []
    labels_pred_val = []
    for batch_x_val, batch_y_val in dataloader_val:
        cavity_model.eval()
        batch_y_pred_val = cavity_model(batch_x_val)
        labels_true_val.append(torch.argmax(batch_y_val, dim=-1).detach().cpu().numpy())
        labels_pred_val.append(
            torch.argmax(batch_y_pred_val, dim=-1).detach().cpu().numpy()
        )
    acc_val = np.mean(
        (np.reshape(labels_true_val, -1) == np.reshape(labels_pred_val, -1))
    )

    print(
        f"Epoch {epoch+1:2d}. Train loss: {loss_running_mean:5.3f}. "
        f"Train Acc: {acc_train:4.2f}. Val Acc: {acc_val:4.2f}"
    )

Epoch  1. Train loss: 2.492. Train Acc: 0.22. Val Acc: 0.11
Epoch  2. Train loss: 1.487. Train Acc: 0.66. Val Acc: 0.22
Epoch  3. Train loss: 0.960. Train Acc: 0.88. Val Acc: 0.25
Epoch  4. Train loss: 0.559. Train Acc: 0.97. Val Acc: 0.24
Epoch  5. Train loss: 0.296. Train Acc: 1.00. Val Acc: 0.23


<IPython.core.display.Javascript object>